In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
import seaborn as sbs
import matplotlib.pyplot as plt

In [2]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 2")
    .config("spark.sql.repl.eagerEval.enabled", True)
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .config("spark.executer.memory", "8g")
    .getOrCreate()
)

22/09/14 00:27:44 WARN Utils: Your hostname, Luo resolves to a loopback address: 127.0.1.1; using 172.17.134.39 instead (on interface eth0)
22/09/14 00:27:44 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/14 00:27:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
sdf = spark.read.parquet("../data/curated/full_data/")
sdf.count()

13614675

In [4]:
clean_sdf = sdf.withColumn('dollar_value', round(F.col('dollar_value'),2))\
    .where(F.col('dollar_value') > 0)\
    .withColumn('log(cost)', log(F.col('dollar_value')))
    

In [5]:
tags = clean_sdf.select('tags').distinct().toPandas()['tags']

In [6]:
import os
output_relative_dir = '../data/curated/full_cleaned_data'
if not os.path.exists(output_relative_dir):
    os.makedirs(output_relative_dir)

In [7]:
for tag in tags:
    revenue = clean_sdf.where(F.col('tags') == tag).toPandas()
    lb =revenue['log(cost)'].quantile(0.025)
    ub =revenue['log(cost)'].quantile(0.975)
    revenue_cleaned = revenue.loc[(revenue['log(cost)'] < ub) & (revenue['log(cost)'] > lb)]
    revenue_cleaned.to_parquet(f'../data/curated/full_cleaned_data/cleaned_{tag}_data.parquet')
    print(f"finished cleaning {tag}")


finished cleaning jewelry


finished cleaning cable


finished cleaning watch


finished cleaning garden supply


finished cleaning antique


...

finished cleaning motor


finished cleaning computer


finished cleaning equipment


finished cleaning gift


finished cleaning bicycle


In [10]:
full_sdf = spark.read.parquet('../data/curated/full_cleaned_data/')
full_sdf.count()

12932766